In [19]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.stats import percentileofscore
import re

In [20]:
# For yolo_nano
# files = [('./analysis/dot_optimization/yolo_nano/without_gaussian_blur/log_file_2.txt', 2), 
#         ('./analysis/dot_optimization/yolo_nano/without_gaussian_blur/log_file_3.txt', 3),
#         ('./analysis/dot_optimization/yolo_nano/without_gaussian_blur/log_file_4.txt', 4),
#         ('./analysis/dot_optimization/yolo_nano/without_gaussian_blur/log_file_5.txt', 5)]
# model_size="nano"

# For yolo_small
# files = [('./analysis/dot_optimization/yolo_small/log_file_2.txt', 2), 
#         ('./analysis/dot_optimization/yolo_small/log_file_3.txt', 3),
#         ('./analysis/dot_optimization/yolo_small/log_file_4.txt', 4),
#         ('./analysis/dot_optimization/yolo_small/log_file_5.txt', 5)]
# model_size="small"

# For yolo_medium
files = [('./analysis/dot_optimization/yolo_medium/log_file_2.txt', 2), 
        ('./analysis/dot_optimization/yolo_medium/log_file_3.txt', 3),
        ('./analysis/dot_optimization/yolo_medium/log_file_4.txt', 4),
        ('./analysis/dot_optimization/yolo_medium/log_file_5.txt', 5)]
model_size="medium"

In [21]:
# --- Data Processing ---
def process_file(filename):
    success_times = []
    try:
        with open(filename, 'r') as f:
            content = f.read()
        entries = re.split(r'\n(?=Processed)', content.strip())
        for entry in entries:
            time_match = re.search(r'Time taken: ([\d.]+)s', entry)
            score_match = re.search(r'Best score: (\d+)', entry)
            if time_match and score_match:
                time = float(time_match.group(1))
                score = int(score_match.group(1))
                if score == 0:
                    success_times.append(time)
    except FileNotFoundError:
        print(f"Warning: File {filename} not found")
    return np.array(success_times)

In [22]:
# --- ECDF Function ---
def ecdf(data):
    """Compute ECDF for a one-dimensional array of data."""
    x = np.sort(data)
    y = np.arange(1, len(x)+1) / len(x)
    return x, y

In [23]:
def plot_ecdf(success_times, num_dots, model_size):
    if len(success_times) == 0:
        print(f"No successful attacks recorded for {num_dots} dots.")
        return
    
    # Ensure Seaborn is applied
    sns.set(style="whitegrid", palette="muted", font_scale=1.2)
    
    # Compute ECDF
    x, y = ecdf(success_times)
    percentiles = [50, 75, 90, 95, 99]
    thresholds = {p: np.percentile(success_times, p) for p in percentiles}
    colors = ['#ff7f0e', '#d62728', '#1f77b4', '#9467bd', '#2ca02c']

    median_time = np.median(success_times)  # Use median instead of mean

    plt.figure(figsize=(10, 6))
    
    # Step plot for ECDF
    plt.step(x, y, where='post', linewidth=2.5, color='#2c7bb6', label=f'{num_dots} Dots ECDF')
    plt.fill_between(x, y, step='post', color='#2c7bb6', alpha=0.2)  # Light blue shade under the curve

    
    # Percentile markers
    for (p, t), c in zip(thresholds.items(), colors):
        y_val = percentileofscore(success_times, t) / 100
        plt.vlines(t, 0, y_val, linestyles='dashed', colors=c, alpha=0.7)
        plt.hlines(y_val, 0, t, linestyles='dashed', colors=c, alpha=0.7)
        plt.scatter(t, y_val, color=c, s=80, edgecolors='black', linewidth=1, zorder=10, label=f'{p}% ({t:.2f}s)')
    
    # Titles and labels
    plt.title(f'Cumulative Distribution for {num_dots} Dots', fontsize=14, fontweight='bold')
    plt.xlabel('Time (seconds)', fontsize=12)
    plt.ylabel('Cumulative Probability', fontsize=12)
    
    # Grid & legend
    plt.grid(True, alpha=0.3, linestyle='dotted')
    plt.legend(title='Percentiles Thresholds:', title_fontsize=10, loc='lower right', frameon=True)
    
    # Ensure directory exists before saving
    save_path = f"ecdf/yolo_{model_size}"
    os.makedirs(save_path, exist_ok=True)
    
    # Save the figure
    save_file = os.path.join(save_path, f"ecdf_{num_dots}_dots.png")
    plt.savefig(save_file, dpi=300, bbox_inches='tight')
    plt.close()  # Close to free memory
    
    print(f"\nStatistics for {num_dots} dots:")
    print(f"Total successful attacks: {len(success_times)}")
    print("Time Thresholds for Percentiles:")
    for p, t in thresholds.items():
        print(f"{p}% of successful attacks complete within {t:.2f}s")
    
    print(f"Plot saved at: {save_file}")

In [24]:
# --- Execute for Each File ---
for filename, num_dots in files:
    success_times = process_file(filename)
    plot_ecdf(success_times, num_dots,model_size)


Statistics for 2 dots:
Total successful attacks: 1024
Time Thresholds for Percentiles:
50% of successful attacks complete within 5.82s
75% of successful attacks complete within 6.06s
90% of successful attacks complete within 6.27s
95% of successful attacks complete within 6.43s
99% of successful attacks complete within 9.91s
Plot saved at: ecdf/yolo_medium\ecdf_2_dots.png

Statistics for 3 dots:
Total successful attacks: 1024
Time Thresholds for Percentiles:
50% of successful attacks complete within 1.65s
75% of successful attacks complete within 8.97s
90% of successful attacks complete within 9.32s
95% of successful attacks complete within 9.62s
99% of successful attacks complete within 11.23s
Plot saved at: ecdf/yolo_medium\ecdf_3_dots.png

Statistics for 4 dots:
Total successful attacks: 1024
Time Thresholds for Percentiles:
50% of successful attacks complete within 1.04s
75% of successful attacks complete within 11.85s
90% of successful attacks complete within 12.30s
95% of succes